In [225]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using CUDA
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
    enable_gpu(false) 
	using Flux: @functor
	using ..Transformers: Abstract3DTensor, Container, epsilon, batchedmul, batched_triu!

end
	

In [226]:
#Labeling: vielleicht ändern

begin
	labels = map(string, 1:10)
	startsym = "11"
	endsym = "12"
	unksym = "0"
	labels = [unksym, startsym, endsym, labels...]
	vocab = Vocabulary(labels, unksym)
end

Vocabulary{String}(13, unk=0)

In [227]:
#function for generate training datas 

#nichts ändern
sample_data() = (d = map(string, rand(1:10, 10)); (d,d))

sample_data (generic function with 1 method)

In [228]:
#nichts ändern

sample_data()

(["7", "10", "9", "4", "8", "7", "9", "4", "9", "7"], ["7", "10", "9", "4", "8", "7", "9", "4", "9", "7"])

In [229]:
#function for adding start & end symbol

#nichts ändern
preprocess(x) = [startsym, x..., endsym]

preprocess (generic function with 1 method)

In [230]:
#nichts ändern

begin
    @show sample_ex = preprocess.(sample_data())
    @show encoded_sample_ex = vocab(sample_ex[1]) #use Vocabulary to encode the training data
    end

sample_ex = preprocess.(sample_data()) = (["11", "4", "7", "6", "4", "10", "9", "10", "10", "5", "5", "12"], ["11", "4", "7", "6", "4", "10", "9", "10", "10", "5", "5", "12"])
encoded_sample_ex = vocab(sample_ex[1]) = [2, 7, 10, 9, 7, 13, 12, 13, 13, 8, 8, 3]


12-element Vector{Int64}:
  2
  7
 10
  9
  7
 13
 12
 13
 13
  8
  8
  3

In [231]:
#nichts ändern
sample = preprocess.(sample_data())

(["11", "1", "10", "5", "2", "1", "8", "9", "2", "9", "10", "12"], ["11", "1", "10", "5", "2", "1", "8", "9", "2", "9", "10", "12"])

In [232]:
#nichts ändern
encoded_sample = vocab(sample[1])

12-element Vector{Int64}:
  2
  4
 13
  8
  5
  4
 11
 12
  5
 12
 13
  3

In [233]:
#define a Word embedding layer which turn word index to word vector

#embeding vielleicht ändern
embed = Embed(512, length(vocab)) |> gpu

Embed(512)

In [234]:
#define a position embedding layer metioned above

#embeding vielleicht ändern
pe = PositionEmbedding(512) |> gpu

PositionEmbedding(512)

In [235]:
#wrapper for get embedding

#embeding vielleicht ändern
function embedding(x)
    we = embed(x, inv(sqrt(512)))
    e = we .+ pe(we)
    return e
  end

embedding (generic function with 1 method)

In [236]:
abstract type AbstractAttention end

In [297]:
create_atten_mask1(T::Type, score::AbstractArray, ::Nothing, future) = create_atten_mask1(T, score, fill!(similar(score, size(score,1), size(score, 2), 1), one(T)), future)
function create_atten_mask1(T::Type, score::AbstractArray, _mask::AbstractArray, future::Bool=false)
  #size(mask) == (q, k, n, b)

  # ql, kl = size(mask)
  mask = copy(_mask)

  maskval = convert(T, -1e9)
  !future && batched_triu!(mask, 0)
  mask .= (1 .- mask) .* maskval
  return mask
end

create_atten_mask1 (generic function with 3 methods)

In [298]:
Flux.@nograd create_atten_mask1

In [300]:
struct MultiheadAttention_1{Q<:Dense, K<:Dense, V<:Dense, O<:Dense, DP<:Dropout} <: AbstractAttention
    head::Int
    future::Bool
    iqproj::Q
    ikproj::K
    ivproj::V
    oproj::O
    drop::DP
end



In [301]:
Flux.functor(mh::MultiheadAttention_1) = (mh.iqproj, mh.ikproj, mh.ivproj, mh.oproj), m -> MultiheadAttention_1(mh.head, mh.future, m..., mh.drop)


In [302]:
"""
    MultiheadAttention(head::Int, is::Int, hs::Int, os::Int;
                       future::Bool=true, pdrop = 0.1)
Multihead dot product Attention Layer, `head` is the number of head, 
`is` is the input size, `hs` is the hidden size of input projection layer of each head, 
`os` is the output size. When `future` is `false`, the k-th token can't see tokens at > k. 
`pdrop` is the dropout rate.
"""

"    MultiheadAttention(head::Int, is::Int, hs::Int, os::Int;\n                       future::Bool=true, pdrop = 0.1)\nMultihead dot product Attention Layer, `head` is the number of head, \n`is` is the input size, `hs` is the hidden size of input projection layer of each head, \n`os` is the output size. When `future` is `false`, the k-th token can't see tokens at > k. \n`pdrop` is the dropout rate.\n"

In [303]:
MultiheadAttention_1(head::Int,
                   is::Int,
                   hs::Int,
                   os::Int;
                   future::Bool=true, pdrop = 0.1) = MultiheadAttention_1(head,
                                                                        future,
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(hs*head, os),
                                                                        Dropout(pdrop),
                                                                        )


function Base.show(io::IO, mh::MultiheadAttention_1)
    hs = div(size(mh.iqproj.weight)[1], mh.head)
    is = size(mh.iqproj.weight)[end]
    os = size(mh.oproj.weight)[1]

    print(io, "MultiheadAttention(")
    print(io, "head=$(mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "$(is)=>$(os)")

    if Flux.istraining()
        print(io, ", dropout=$(mh.drop.p))")
    else
        print(io, ")")
    end
end

In [304]:
function (mh::MultiheadAttention_1)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: Abstract3DTensor{T},
                         A2 <: Abstract3DTensor{T},
                         A3 <: Abstract3DTensor{T}}
qs = size(query)
ks = size(key)
vs = size(value)

#size(ipq) == (h, q_seq_len, batch)
ipq = @toNd mh.iqproj(query)
ipk = @toNd mh.ikproj(key)
ipv = @toNd mh.ivproj(value)

h = size(ipq, 1)
hs = div(h, mh.head)

#size(ipq) == (hs, q_seq_len, head, batch)
ipq = permutedims(reshape(ipq, hs, mh.head, qs[2], qs[3]), [1, 3, 2, 4])
ipk = permutedims(reshape(ipk, hs, mh.head, ks[2], ks[3]), [1, 3, 2, 4])
ipv = permutedims(reshape(ipv, hs, mh.head, vs[2], vs[3]), [1, 3, 2, 4])

#size(ipq) == (hs, q_seq_len, head * batch)
ipq = reshape(ipq, hs, qs[2], :)
ipk = reshape(ipk, hs, ks[2], :)
ipv = reshape(ipv, hs, vs[2], :)

atten = attention(ipq,ipk,ipv,
mask,
mh.future,
mh.drop)

atten = permutedims(reshape(atten, hs, qs[2], mh.head, qs[3]), [1, 3, 2, 4]) #size(atten) == (hs, head, ql, b)
atten = reshape(atten, h, qs[2], qs[3]) #size(atten) == (h, ql, b)

out = @toNd mh.oproj(atten)
out #size(out) == (h, q_seq_len, batch)
end

function (mh::MultiheadAttention_1)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: AbstractMatrix{T},
                         A2 <: AbstractMatrix{T},
                         A3 <: AbstractMatrix{T}}

# size(query) == (dims, seq_len)
ipq = mh.iqproj(query)
ipk = mh.ikproj(key)
ipv = mh.ivproj(value)

h = size(ipq)[1] #h == hs * head
hs = div(h, mh.head)

#size(hq) == (hs, seq_len, head)
hq = permutedims(reshape(ipq, hs, mh.head, :), [1, 3, 2])
hk = permutedims(reshape(ipk, hs, mh.head, :), [1, 3, 2])
hv = permutedims(reshape(ipv, hs, mh.head, :), [1, 3, 2])

atten = attention(hq, hk, hv,
mask,
mh.future,
mh.drop)

# size(atten) == (head*hs, seq_len)
atten = reshape(permutedims(atten, [1, 3, 2]), h, :)

mh.oproj(atten)
end



In [305]:
function apply_mask1(score, mask)
    s = size(score)
    ms = size(mask)
    bxn = s[end]
    b = ms[end]
    if bxn == b || b == 1
      return score .+ mask
    else
      return reshape(reshape(score, s[1:end-1]..., :, b) .+
                     reshape(mask, ms[1:end-1]..., 1, b), s)
    end
  end
  
  apply_mask1(score::AbstractArray{T}, ::Nothing, future) where T = future ? score : apply_mask1(score, create_atten_mask1(T, score, nothing, future))
  apply_mask1(score::AbstractArray{T}, mask, future) where T = apply_mask(score, create_atten_mask1(T, score, mask, future))
  

apply_mask1 (generic function with 3 methods)

In [306]:
function attention(query::A1,
    key::A2,
    value::A3,
    mask, future::Bool,
    dropout) where {T,
                    A1 <: Abstract3DTensor{T},
                    A2 <: Abstract3DTensor{T},
                    A3 <: Abstract3DTensor{T}}
#size(query) == (dims, {q,k}_seq_len, batch) == size(key) == size(value)
#size(score) == (k_seq_len, q_seq_len, batch)
dk = size(key, 1)
score = batchedmul(key, query; transA = true)
score = score ./ convert(T, sqrt(dk))

score = apply_mask1(score, mask, future)
score = softmax(score; dims=1)
dropout !== nothing && (score = dropout(score))
batchedmul(value, score) #size(return) == (dims, q_seq_len, batch)
end

attention (generic function with 1 method)

In [307]:
abstract type AbstractTransformer end

In [308]:
struct PwFFN{Di<:Dense, Do<:Dense}
    din::Di
    dout::Do
end

@functor PwFFN


"just a wrapper for two dense layer."
PwFFN(size::Int, h::Int, act = relu) = PwFFN(  #relu vielleicht auf elu + 1 oder nur elu ändern
    Dense(size, h, act),
    Dense(h, size)
)

function (pw::PwFFN)(x::AbstractMatrix)
  # size(x) == (dims, seq_len)
  pw.dout(pw.din(x))
end

function (pw::PwFFN)(x::A) where {T, N, A<:AbstractArray{T, N}}
  new_x = reshape(x, size(x, 1), :)
  y = pw(new_x)
  return reshape(y, Base.setindex(size(x), size(y, 1), 1))
end

In [309]:
struct Transformer1{MA<:MultiheadAttention_1, LA<:LayerNorm, P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
    mh::MA
    mhn::LA
    pw::P
    pwn::LP
    drop::DP
end

@functor Transformer1


"""
    Transformer(size::Int, head::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)
    Transformer(size::Int, head::Int, hs::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)  

Transformer layer.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
When `future` is `false`, the k-th token can't see the j-th tokens where j > k. `pdrop` is the dropout rate.
"""

########
#ganze Funktion von Transformer ändern
########
function Transformer1(size::Int, head::Int, ps::Int; future::Bool = true, act = relu, pdrop = 0.1)  #relu vielleicht wieder auf elu ändern
    rem(size, head) != 0 && error("size not divisible by head")
    Transformer1(size, head, div(size, head), ps;future=future, act=act, pdrop=pdrop)
end

Transformer1(size::Int, head::Int, hs::Int, ps::Int; future::Bool = true, act = relu, pdrop = 0.1) = Transformer1(
    MultiheadAttention_1(head, size, hs, size; future=future, pdrop=pdrop),
    LayerNorm(size),
    PwFFN(size, ps, act),
    LayerNorm(size),
    Dropout(pdrop),     #braucht man das? vielleicht nicht
)

function (t::Transformer1)(x::A, mask=nothing) where {T, N, A<:AbstractArray{T, N}}
    dropout = t.drop
    a = t.mh(x, x, x; mask=mask)
    a = dropout(a)
    res_a = x + a
    res_a = t.mhn(res_a)
    pwffn = t.pw(res_a)
    pwffn = dropout(pwffn)
    res_pwffn = res_a + pwffn
    res_pwffn = t.pwn(res_pwffn)
    res_pwffn
end

function Base.show(io::IO, t::Transformer1) #zum visualisieren --> nice to have
    hs = div(size(t.mh.iqproj.weight)[1], t.mh.head)
    h, ps = size(t.pw.dout.weight)

    print(io, "Transformer(")
    print(io, "head=$(t.mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "pwffn_size=$(ps), ")
    print(io, "size=$(h)")
    if Flux.istraining()
        print(io, ", dropout=$(t.drop.p))")
    else
        print(io, ")")
    end
end


In [310]:
###########
    #auch diese Funktion ändern
    ###############
    struct TransformerDecoder1{MA<:MultiheadAttention_1, LA<:LayerNorm,
        IMA<:MultiheadAttention_1, ILA<:LayerNorm,
        P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
mh::MA
mhn::LA
imh::IMA
imhn::ILA
pw::P
pwn::LP
drop::DP
end

@functor TransformerDecoder1

"""
TransformerDecoder(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
TransformerDecoder(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1)

TransformerDecoder layer. Decode the value from a Encoder.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
`pdrop` is the dropout rate.
"""
function TransformerDecoder1(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
rem(size, head) != 0 && error("size not divisible by head")
TransformerDecoder1(size, head, div(size, head), ps; act=act, pdrop=pdrop)
end

TransformerDecoder1(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1) = TransformerDecoder1(
MultiheadAttention_1(head, size, hs, size; future=false, pdrop=pdrop),
LayerNorm(size),
MultiheadAttention_1(head, size, hs, size; future=true, pdrop=pdrop), #future = true --> Unterschied zu oben??
LayerNorm(size),
PwFFN(size, ps, act),
LayerNorm(size),
Dropout(pdrop),
)

function (td::TransformerDecoder1)(x::AbstractArray{T,N}, m, mask=nothing) where {T,N}
dropout = td.drop
a = td.mh(x,x,x)
a = dropout(a)
res_a = x + a
res_a = td.mhn(res_a)

ia = td.imh(res_a, m, m, mask=mask)
ia = dropout(ia)
res_ia = res_a + ia
res_ia = td.imhn(res_ia)

pwffn = td.pw(res_ia)
pwffn = dropout(pwffn)
res_pwffn = res_ia + pwffn
res_pwffn = td.pwn(res_pwffn)
res_pwffn
end

function Base.show(io::IO, td::TransformerDecoder1)
hs = div(size(td.imh.iqproj.weight)[1], td.imh.head)
h, ps = size(td.pw.dout.weight)

print(io, "TransformerDecoder(")
print(io, "head=$(td.mh.head), ")
print(io, "head_size=$(hs), ")
print(io, "pwffn_size=$(ps), ")
print(io, "size=$(h)")
if Flux.istraining()
print(io, ", dropout=$(td.drop.p))")
else
print(io, ")")
end
end

In [311]:
#define 2 layer of transformer

#layer auf jeden Fall ändern
encode_t1 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [312]:
encode_t2 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [313]:
encode_t3 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [314]:
encode_t4 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [315]:
#define 2 layer of transformer decoder

#layer auf jeden Fall ändern
decode_t1 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [316]:
decode_t2 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [317]:
decode_t3 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [318]:
decode_t4 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [319]:
#define the layer to get the final output probabilities

#ÄNDERN
linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

Positionwise(Dense(512, 13), logsoftmax)

In [320]:
#neue Layer einfügen
function encoder_forward(x)
    e = embedding(x)
    t1 = encode_t1(e)
    t2 = encode_t2(t1)
    t3 = encode_t2(t2)
    t4 = encode_t2(t3)
    return t2
  end

encoder_forward (generic function with 1 method)

In [321]:
#neue Layer einfügen
function decoder_forward(x, m)
    e = embedding(x)
    t1 = decode_t1(e, m)
    t2 = decode_t2(t1, m)
    t3 = decode_t3(t2, m)
    t4 = decode_t4(t3, m)
    p = linear(t4)
    return p
  end

decoder_forward (generic function with 1 method)

In [322]:
#bleibt so
enc = encoder_forward(encoded_sample)

512×12 Matrix{Float32}:
  0.468389  -0.239751   -0.781162  …  -0.105302   0.371754   0.732726
  0.625913   0.814828    0.276023      0.26693   -0.221515  -0.239351
  0.786205   0.0389965  -0.453369     -0.109634   0.207264   0.981548
  0.558399   0.457303   -0.205462      0.203294  -0.330861  -0.301439
 -0.499682  -1.24878    -1.74873      -1.56921   -1.52691   -0.971043
 -1.37978   -1.29783    -1.72479   …  -1.22446   -1.82067   -2.08222
 -0.627585  -1.32785    -1.76623      -1.43402   -1.37365   -0.915869
  1.21342    1.18821     0.703239      1.48738    1.01276    0.644496
 -0.612075  -1.04587    -1.44484      -1.47799   -1.45016   -1.08645
  3.12137    3.34021     2.92903       3.13671    2.57095    2.10821
  ⋮                                ⋱              ⋮         
 -0.377055  -0.409071   -0.31353       0.100499   0.174365   0.170537
  0.754467   0.782147    0.839845      1.12181    0.96634    1.1155
 -0.729594  -0.631009   -0.49565   …  -0.550436  -0.411324  -0.634024
 -0.732477

In [323]:
#bleibt so
probs = decoder_forward(encoded_sample, enc)

13×12 Matrix{Float32}:
 -3.44137   -3.44499   -3.45885   -3.48929   …  -3.53358  -3.60441  -3.65591
 -5.03886   -5.02966   -4.96638   -4.93876      -4.99748  -5.0417   -5.04876
 -4.72285   -4.70291   -4.71861   -4.69051      -4.41509  -4.45054  -4.43333
 -2.95765   -2.98813   -2.99307   -2.99243      -2.85001  -2.89214  -2.92647
 -3.3846    -3.44186   -3.44122   -3.48171      -3.14828  -3.172    -3.21538
 -2.32813   -2.30568   -2.24178   -2.17152   …  -2.01842  -1.95773  -1.87893
 -2.40015   -2.46888   -2.53813   -2.59019      -2.65999  -2.6486   -2.64615
 -3.1859    -3.2162    -3.23506   -3.26898      -3.14254  -3.0641   -3.00239
 -3.11701   -3.13662   -3.12898   -3.10877      -2.96538  -3.01154  -3.02196
 -2.81244   -2.81068   -2.78788   -2.74565      -2.7592   -2.71611  -2.64822
 -2.33343   -2.36884   -2.45145   -2.50322   …  -2.37291  -2.43201  -2.48044
 -2.89845   -2.87283   -2.80676   -2.76297      -2.54701  -2.54185  -2.55631
 -0.965391  -0.936751  -0.931517  -0.938103     -1.14

In [324]:
#hier vielleicht etwas ändern --> herausfinden, was diese funktion macht
function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

smooth (generic function with 1 method)

In [325]:
Flux.@nograd smooth

In [326]:
#define loss function

#ÄNDERN
function loss(x, y)
    label = onehot(vocab, y) #turn the index to one-hot encoding
    label = smooth(label) #perform label smoothing
    enc = encoder_forward(x)
    probs = decoder_forward(y, enc)
    l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])  #erstmal so lassen
    return l
  end

loss (generic function with 1 method)

In [327]:
#collect all the parameters

#Layer neu einfügen
ps = params(embed, pe, encode_t1, encode_t2, encode_t3, encode_t4, decode_t1, decode_t2, decode_t3, decode_t4, linear)

Params([Float32[0.19031276 -1.2930133 … -0.735448 0.57854205; -1.6189741 -1.0117838 … -0.41122213 0.6526874; … ; -0.47337812 -1.1268125 … -0.69679254 0.024595367; 0.27807164 0.569649 … 0.73205245 0.4195867], Float32[0.5403023 -0.41614684 … 0.4000682 0.9873536; 0.8218562 0.9364147 … 0.37902638 0.97646356; … ; 1.0 1.0 … 0.9943822 0.99437124; 0.0001 0.0002 … 0.10212166 0.10222114], Float32[0.042281516 -0.01930578 … 0.017422624 -0.03029542; -0.028011655 0.035027128 … 0.06337698 0.071344815; … ; -0.014897446 -0.076423936 … 0.003734632 -0.064617805; 0.016447447 0.005082075 … 0.05951089 -0.023200577], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[-0.06712648 -0.006214203 … 0.031238861 0.025760138; 0.07226734 0.041060966 … -0.057337847 -0.0021938835; … ; 0.030360281 0.012778735 … 0.073901385 -0.057870988; 0.002712333 0.06016846 … -0.03696688 0.062364936], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 

In [328]:
opt = ADAM(1e-4) #bleibt

ADAM(0.0001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [329]:
#define training loop

#ÄNDERN siehe Paper Kapitel 4.1
function train!()
    @info "start training"
    for i = 1:1000
      data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
      x, y = preprocess.(data[1]), preprocess.(data[2])
      x, y = vocab(x), vocab(y) #encode the data
      x, y = todevice(x, y) #move to gpu
      grad = gradient(()->loss(x, y), ps)
      if i % 8 == 0
          l = loss(x, y)
          println("loss = $l")
      end
      update!(opt, ps, grad)
    end
  end

train! (generic function with 1 method)

In [330]:
train!()

┌ Info: start training
└ @ Main /Users/johannes/Documents/GitHub/DM2022-LinearTransformers/test/Duplicat_Exercise11.ipynb:5


loss = 98.160194


loss = 82.9769


loss = 80.39799


loss = 77.87157


...

loss = 0.009129214


loss = 0.009761279


loss = 0.0046296967


loss = 0.0031574233


loss = 0.0061669056


In [331]:
#erst einmal so lassen, vielleicht einzelne Änderungen
function translate(x)
    ix = todevice(vocab(preprocess(x)))
    seq = [startsym]

    enc = encoder_forward(ix)

    len = length(ix)
    for i = 1:2len
        trg = todevice(vocab(seq))
        dec = decoder_forward(trg, enc)
        #move back to gpu due to argmax wrong result on CuArrays
        ntok = onecold(collect(dec), labels)
        push!(seq, ntok[end])
        ntok[end] == endsym && break
    end
  seq[2:end-1]
end

translate (generic function with 1 method)

In [332]:
translate(map(string, [5,5,6,6,1,12,3,4,6]))

10-element Vector{String}:
 "5"
 "5"
 "6"
 "6"
 "1"
 "7"
 "3"
 "4"
 "6"
 "6"